# Lecture 7 - Matrix Inverse, Iterative Methods 👾

**Learning Objectives**
* Compute the matrix inverse using LU Decomposition.
* Interpret the inverse matrix to determine the system's response to external forcings
* Check the stability of our system by computing the condition number and determining the number of suspect digits.
* Check if a matrix is diagonally dominant and implement the Gauss-Seidel methods for solving a system of linear equations.

## Calculating/Interpreting Inverse Coefficients

**C&C Case Study 12.1: System of 5 Reactors with Unknown Concentrations**

<p align="center">
  <img src="https://github.com/cdefinnda/ECI-115_HW-Images/blob/main/cha32077_1203.png?raw=true" alt="cha32077_1203" width=500>
</p>

>* Reactor 1: $Q_{01}c_1+Q_{31}c_3=Q_{12}c_1+Q_{15}c_1 \rightarrow 6c_1-c_3=50$
* Reactor 2: $Q_{12}c_1=Q_{25}c_2+Q_{24}c_2+Q_{23}c_2 \rightarrow -3c_1+3c_2=0$
* Reactor 3: $-c_2+9c_3=160$
* Reactor 4: $-c_2-8c_3+11c_4-2c_5=0$
* Reactor 5: $-3c_1-c_2+4c_5=0$

### Set up Linear System (Same as L06 Coding Notebook)

In [ ]:
# Import Libraries
import numpy as np
import scipy.linalg as sl

# Set Up Lineary System
A = np.array([[6, 0, -1, 0, 0],
              [-3, 3, 0, 0, 0],
              [0, -1, 9, 0, 0],
              [0, -1, -8, 11, -2],
              [-3, -1, 0, 0, 4]])

b = np.array([50, 0, 160, 0, 0]).T  # Numpy will assume the dimensions of a 1D
                                    # array so that matrix math works out.

n = A.shape[0]                      # Store # of rows of A for LU Decomposition

# Convert all elements of A and b to float
A = A.astype(float)
b = b.astype(float)

# Check if Matrix A is singular (i.e., det = 0)
#print(sl.det(A))   # We know from last time that A is not singular

### Perform LU Decomposition

In [ ]:
# LU Decomposition
# Note: This does not include partial pivoting and may encounter divide by zero errors
# See C&C Figure 10.2 for pseudocode for LU with partial pivoting
L = np.eye(n)
U = A.copy()

for k in range(n-1):
    for i in range(k+1,n):
        L[i,k] = U[i,k] / U[k,k] # Multiplication Factor
        U[i,:] = U[i,:] - L[i,k] * U[k,:] # Modify row i based on pivot row k

### Solve for Matrix Inverse using LU Decomposition

By definition, $A$ and $A^{-1}$ have the following relationship:
$$
AA^{-1}=I=
\begin{bmatrix}
1 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 1
\end{bmatrix}
$$

To solve for $A^{-1}$, we can break the matrix up into the following column vectors:

$$A^{-1}=[x_1\   x_2\   x_3\   x_4\   x_5]$$

Where:
>$$
Ax_1=
\begin{bmatrix}
1\\
0\\
0\\
0\\
0
\end{bmatrix}
$$
>
>$$
Ax_2=
\begin{bmatrix}
0\\
1\\
0\\
0\\
0
\end{bmatrix}
$$
>
>and so on...

By breaking up $A^{-1}$ into individual column vectors ($x_1, x_2, x_3, x_4, x_5$), we can use the LU Decomposition of $A$ to solve for $x_1$, $x_2$, $x_3$, $x_4$ and $x_5$ individually, by first solving for $d_1$ by forward substitution:

$$
L d_1 = Ax_1
$$

And then solving for $x_1$ by back substitution:

$$
U x_1 = d_1
$$

This process can then be repeated for the all other columns to obtain $x_2, x_3, x_4,$ and $x_5$.

Since this process requires repeated forward/back substitution, it helps to define a function that does this for us automatically:

In [ ]:
# Inputs: L, U, b
# Outputs: x such that LUx = b
def forward_back_sub(L, U, b):

    # forward substitution
    d = np.zeros(n)
    d[0] = b[0] / L[0,0]

    for i in range(1,n):
        d[i] = (b[i] - L[i,:] @ d) / L[i,i]

    # back substitution
    x = np.zeros(n)
    x[-1] = d[-1] / U[-1,-1] # index -1 for last element

    for i in range(n-2,-1,-1): # loop backward starting from second-to-last row
        x[i] = (d[i] - U[i,i+1:n] @ x[i+1:n]) / U[i,i]

    return x

With this function defined, we can compute $\mathbf{A^{-1}}$ using the function iterate through the columns of an identity matrix $\mathbf{I_j}$ in order to comput the corresponding columns of $\mathbf{A^{-1}_j}$.

Note: The original $\mathbf{b}$ vector is not used here. The inverse only depends on $\mathbf{A}$.

💪 Complete the code below:

In [ ]:
# Solve for inverse using RHS columns from I
I = np.eye(n)
Ainv = np.zeros((n,n))

for j in range(n):
    # [Fill in code here to solve for the columns of Ainv using foward_back_sub().]

np.set_printoptions(precision=3, suppress=True)
print(Ainv)


print('\n Compare to SciPy function (sl.inv(A)):', np.allclose(Ainv, sl.inv(A)))

### Interpreting the Matrix Inverse ($\mathbf{A^{-1}}$)

The coefficients from the inverse help us interpret the relationship between the forcing and the unknown system states. Last time we changed the input forcing $c_{01}=10$ to $c_{01}=20$ and solved again to find the unknown states. Here we can see that inverse coefficients $a_{ij}^{-1}$ represent the change in state variable $i$ per unit change in forcing $j$.

This change in $c_{01}$ causes the first forcing element, $b_1 = Q_{01}c_{01}$, to change 50 units (note $Q_{01}=5$ is unchanged). So, we would expect the unknown $c_2$ to change by $0.17 (50) = 8.5$ units. The change in $c_3$ would be $0.019 (50) = 0.95$ units.

This interpretation depends on the order of the equations in the original matrix, which is arbitrary depending how the state variables are numbered.



Last time, we calculated the change in our steady state concentrations when our influent concentrations changed from $c_{01}=10 \rightarrow 20$ and $c_{03}=20 \rightarrow 10$, corresponding with concentration loads changing from $b_1 = Q_{01}c_{01}= 50 \rightarrow 100$ ($Q_{01}=5$ is unchanged) and $b_3 = Q_{03}c_{03}= 160 \rightarrow 80$ ($Q_{03}=8$ is unchanged). The resulting steady state concentrations became:

* $c_1 = 11.509 \rightarrow 18.491$
* $c_2 = 11.509 \rightarrow 18.491$
* $c_3 = 19.057 \rightarrow 10.943$
* $c_4 = 16.998 \rightarrow 13.002$
* $c_5 = 11.509 \rightarrow 18.491$

The inverse matrix $\mathbf{A^{-1}}$ can actually provide insights into the relative contributions of these loadings to the changes in steady state concentration. In $\mathbf{A^{-1}}$, each entry $a^{-1}_{ij}$ tells us how much the $i^{th}$ state variable (i.e., $c_i$) changes per 1-unit change in the $j^{th}$ loading input (i.e., $b_j$).

Let's focus on $c_4$:
>The change in $b_1$ is $\Delta b_1=50$. To see the effect this would have, on $c_4$, we look to $a^{-1}_{4,1} = 0.06$. Based on this value, we would expect $\Delta c_{4,1} = a^{-1}_{4,1}(\Delta b_1)=+3$.

>Additionally the change in $b_3$ is $\Delta b_1=-80$. To see the effect this would have, on $c_4$, we look to $a^{-1}_{4,3} = 0.087$. Based on this value, we would expect $\Delta c_{4,3} = a^{-1}_{4,3}(\Delta b_3)=-6.96$.

>When we combine these changes, we get: $\Delta c_4 = \Delta c_{4,1} + \Delta c_{4,3} = -3.96$, which corresponds with the new steady state concentration we see after the change in influent concentrations: $\Delta c_4 = c_1^{final} - c_1^{initial} = 13.002 - 16.998=-3.996$ (where differences arise because of rounding).

This relationship gives us the general formula for how changes in load impact the steady state values of state variables:

$$\Delta x_{ij}=a^{-1}_{ij} \Delta b_j$$

The above formula is useful for determining the relative contribution of different changes in loadings to the total changes observed in the state variable.

### Condition Number

The condition number of a matrix $\mathbf{A}$ is:

$$\kappa(\mathbf{A})=||\mathbf{A}|| \cdot ||\mathbf{A^{-1}}||$$

Where $||\mathbf{A}||$ indicates the norm of $\mathbf{A}$, which is a non-negative scalar that represents the magnitude and size of $\mathbf{A}$. There are various ways to take the matrix norm (e.g., 2-Norm, column-sum, Frobenius), which would impact how $\kappa(\mathbf{A})$ is calculated.

The condition number provides information about how much we can trust our solution based on the stability of our system, how error becomes amplified, numerical reliability, and digit loss impacting precision. In general:

* $\kappa(\mathbf{A})$ ~ $10^1$ to $10^3$: System is stable and accurate.
* $\kappa(\mathbf{A})$ ~ $10^3$ to $10^6$: System is sensitive to errors.
* $\kappa(\mathbf{A})$ > $10^8$: System is ill-conditioned.
* $\kappa(\mathbf{A})$ = $\infty$: System is singular (not invertible).

In [ ]:
k_1 = np.linalg.cond(A, 'fro') # Use the Frobenius (Euclidean) norm
print(k_1)

In [ ]:
k_2 = sl.norm(A) * sl.norm(Ainv) # Should match
print(k_2)

Because the condition number is low. We are not concerned about a singular matrix.

See below for an example of an ill-conditioned matrix. The closer the value `1.00001` gets to 1, the higher the condition number will be, and the fewer digits of precision we will be able to trust in the result.

In [ ]:
r = 0.00001
M = np.array([[10, 1], [10, 1+r]])
M_cond = np.linalg.cond(M, 'fro')

print(M_cond)

If we want to estimate the number of suspect digits (recall that numbers in Google Colab retain ~16 digits of precision) we can calculate this by taking the the log of our condition number:

$$Suspect\ Digits = log_{10}(\kappa(\mathbf{A}))$$



In [ ]:
sus_digits = np.log10(M_cond)

print(sus_digits) # Output: ~6. Indicates we should only trust ~10 sig figs.

❓ Adjust `r` to see the impact on the number of suspect digits. How does decreasing `r` by an order of magnitude change the number of suspect digits?

* [???]

## Gauss-Seidel Method

The Gauss-Seidel Method is an interative approach used to solve large systems of linear equations. This technique can be more efficient than LU Decomposition, especially if our matrix $\mathbf{A}$ is sparse and diagonally-dominant.

### Determining if a Matrix is Diagonally Dominant

A matrix is diagonally dominant if, for every row, the magnitude of the main diagonal is greater than the sum of the magnitudes of all the other entries in that row.

This is important because it will tell us if the Gauss-Seidel method will converge.

In [ ]:
def is_diagonally_dominant(A):
    diag = np.diag(np.abs(A))             # Extracts the abs values of the diagonal elements.
    rowsum = np.sum(np.abs(A), axis=1)    # Computes the sum of the abs values across each row.
    return np.all(diag > rowsum - diag)   # Compares diagonal element to the row sum (- corresponding diagonal element).
                                          # Will output False if any row doesn't meet this condition.
is_diagonally_dominant(A)

Unfortunately, our matrix $\mathbf{A}$ is not going to work with the Gauss-Seidel method:
$$
\mathbf{A}=
\begin{bmatrix}
6 & 0 & -1 & 0 & 0\\
-3 & 3 & 0 & 0 & 0\\
0 & -1 & 9 & 0 & 0\\
0 & -1 & -8 & 11 & -2\\
-3 & -1 & 0 & 0 & 4
\end{bmatrix}
$$

\
Let's use the another matrix from the textbook instead to illustrate how the Gauss-Seidel method works:
$$
\mathbf{A}=
\begin{bmatrix}
8 & 3 & -3\\
-2 & -8 & 5\\
3 & 5 & 10
\end{bmatrix}
$$


In [ ]:
# Reset variables
%reset -f
import numpy as np
import scipy.linalg as sl


# Re-define A and b
A = np.array([[8, 3, -3], [-2, -8, 5], [3, 5, 10]])
b = np.array([14, 5, -8]).T

# Re-define is_diagonally_dominant function
def is_diagonally_dominant(A):
    diag = np.diag(np.abs(A))             # Extracts the abs values of the diagonal elements.
    rowsum = np.sum(np.abs(A), axis=1)    # Computes the sum of the abs values across each row.
    return np.all(diag > rowsum - diag)   # Compares diagonal element to the row sum (- corresponding diagonal element).
                                          # Will output False if any row doesn't meet this condition.


# Compute whether new A is diagonally dominant
is_diagonally_dominant(A)

## Gauss-Seidel Iteration

Recall that the Gauss-Seidel initializes our state variables $\mathbf{x}$ as an array of zeros (iteration = $k$). Then, each  these values are updated by successively solving each equation with the previously values for $\mathbf{x}$ and immediately updating these values based on the result:

$$x_1^{(k+1)}=\frac{1}{a_{11}} \big(b_1 - a_{12}x_2^{(k)}- a_{13}x_3^{(k)} \big)$$

$$x_2^{(k+1)}=\frac{1}{a_{22}} \big(b_2 - a_{21}x_1^{(k+1)}- a_{23}x_3^{(k)} \big)$$

$$x_3^{(k+1)}=\frac{1}{a_{33}} \big(b_3 - a_{31}x_1^{(k+1)}- a_{32}x_2^{(k+1)} \big)$$


Note: The error is defined by the norm of $\mathbf{A}\mathbf{x} - \mathbf{b}$. Loop until tolerance `1e-8`.

💪 In the `for` loop complete the code for re assigning values for $\mathbf{x_i}$:

In [ ]:
n = 3
x = np.zeros(n)
err = 9999
errors = []
num_iter = 0

while err > 1e-8:
    for i in range(n): # loop over variables
        x[i]  = # [Fill in code here.]

    err = sl.norm(A @ x - b, 2) # The "2" specifies taking the Euclidean vector norm
    errors.append(err) # Store these for plotting
    num_iter += 1

# Print Result
print(x)

In [ ]:
# Plot Convergence
import matplotlib.pyplot as plt
plt.semilogy(errors) # Plots a semi-log plot, where the y-axis is in log scale.
plt.xlabel('Iterations')
plt.ylabel('Log Error (norm)')
plt.show()

Each iteration, $n$ variables are updated with a total of $n$ arithmetic operations each. If $k$ iterations are performed, the total operation count scales as $O(kn^2)$. Compared to Gauss elimination $O(n^3)$, this iterative method may be helpful in larger systems of equations where $k < n$. However, that is not the case here, where $n=3$ and $k=26$ iterations are needed to reach a tolerance of `1e-8`.